## 準備
RAGエージェントを作成するには次の3つの要素が必要となる。

- model
- embedding
- vector store


### model

In [ ]:
from transformers import BitsAndBytesConfig
# HuggingFaceのモデルを利用するため
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    model_kwargs={"quantization_config": bnb_config},
)



### embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

### vector store

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## Indexing

Indexingは次の流れとなっている。
- 文書の読み込み
- 文書の分割
- Store

### 文書の読み込み

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader(
    web_paths=("https://example.com/",),
)

In [ ]:
documents = loader.load()

In [ ]:
print(len(documents[0].page_content))

### 文書の分割

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 30,
    add_start_index = True
)

In [ ]:
data_split = text_splitter.split_documents(documents)


### Store

In [ ]:
ids = vector_store.add_documents(documents=data_split)


In [ ]:
ids

## Retrieval と Generation

In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrive(query):
    """情報の検索"""
    relventa_docs = vector_store.similarity_search(query,k=3)
    serialized = "\n\n".join(
        (f"Metadata: {doc.metadata}\nContent: {doc.page_content}")
        for doc in relventa_docs
    )
    return serialized, relventa_docs

In [ ]:
from langchain.agents import create_agent

tools = [retrive]

# System Prompt
system_prompt = (
    "あなたは役立つチャットボットです。"
    "質問に答える際、必要に応じて retrive ツールを絶対に使用して情報を検索してください。"
    "回答は日本語で行ってください。"
)


chat_model = ChatHuggingFace(llm=llm)
chat_model.bind_tools(tools)

agent = create_agent(chat_model,tools,system_prompt=system_prompt)

In [ ]:
query = (
    "文章の目的を教えてください"
)

In [ ]:
for event in agent.stream(
    {"messages":[{"role":"user","content":query}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

